<a href="https://colab.research.google.com/github/treefield00/Calc/blob/main/Calc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 四則演算と剰余と冪乗の混在する式の計算を逆ポーランド記法で実装
# +：加算、-：減算、*：乗算、/：除算、%：剰余、^：冪乗


In [35]:
def operate(left_operand: 'float', right_operand: 'float', operator: 'str') -> 'float':
    """二項演算子による計算の関数

    :param left_operand: 二項演算の左の被演算子
    :type left_operand: float
    :param right_operand: 二項演算の右の被演算子
    :type right_operand: float
    :return: 演算結果
    :rtype: float
    """

    #確認のためprint
    print('二項演算', left_operand, operator, right_operand)

    try:
        if operator == "+" :
            return left_operand + right_operand
        if operator == "-" :
            return left_operand - right_operand
        if operator == "*" :
            return left_operand * right_operand
        if operator == "/" :
            return left_operand / right_operand
        if operator == "%" :
            return left_operand % right_operand
        if operator == "^" :
            return left_operand ** right_operand
        raise Exception("演算子が不正です。")
    
    except Exception as exc:
        print("二項演算で例外が発生しました。", str(left_operand) + operator + str(right_operand), exc)
        raise



In [ ]:
operate(.2,0,"/")


In [37]:
#トークン用クラス
class Token:
    """トークン格納用のクラス

    :Attribute letter: 式のトークン
    :type letter: str
    :Attribute order: トークンの優先順位
    :type order: int
    """

    def __init__(self, letter, order):
        self.letter = letter
        self.order = order


In [38]:
def get_token_list(expression: 'str')-> 'list':
    """トークンのリストを出力する関数
    
    演算子、数値、括弧をトークンごとに配列に格納する
    同時にそれぞれの優先順位も配列に格納する
    優先順位
    5  (
    4  0〜9 .
    3  ^
    2  * / %
    1  + -
    0  )
    -1  番兵

    :param expression: 式の文字列列
    :type expression: str
    :return: Tokenクラスの配列
    :rtype: list
    """

    #
    i = 0

    #トークン格納用の配列
    TokenRecord = []
    TokenRecord.append(Token('',-1)) #番兵

    for lttr in expression:

        #lttrが数値
        if lttr in '0123456789.':
            #直前が+-
            if TokenRecord[i].order == 1:
                #二つ前が(^*/%または配列の最初
                if TokenRecord[i-1].order in (5,3,2,-1):
                    #+-と連結し数値につく符号とする
                    TokenRecord[i].letter = TokenRecord[i].letter + lttr
                    TokenRecord[i].order = 4
                #二つ前が数値または ）
                elif TokenRecord[i-1].order in (4,0):
                    if TokenRecord[i-1].letter[-1] == '.':
                        raise Exception('「 ' + TokenRecord[i].letter + ' 」の直前に小数点がきています。')
                    #トークンに分ける
                    i += 1
                    TokenRecord.append(Token(lttr,4))
                else:
                    raise Exception('+ または - が連続しています。')
            #直前が+-以外の演算子・括弧 (
            elif TokenRecord[i].order in (5,3,2,-1):
                #数値のトークン
                i += 1
                TokenRecord.append(Token(lttr,4))
            #直前が括弧 ）
            elif TokenRecord[i].order == 0:
                raise Exception('「 ）」の直後に数値がきています。')
            #直前が数値
            elif TokenRecord[i].order == 4:
                if lttr == '.':
                    if '.' in TokenRecord[i].letter:
                        raise Exception('小数点「 . 」の使い方が不正です。')
                else:
                    if TokenRecord[i].letter[0] == '0' and TokenRecord[i].letter[1] != '.':
                        raise Exception('整数部の先頭に「0」があります。')
                #同一トークンとして連結
                TokenRecord[i].letter = TokenRecord[i].letter + lttr
        #Lttrが（　括弧
        elif lttr == '(':
            #直前が+-
            if TokenRecord[i].order == 1:
                #二つ前が(^*/%または配列の最初
                if TokenRecord[i-1].order in (5,3,2,-1):
                    #+-を符号にするため乗算に変換
                    TokenRecord[i].letter = TokenRecord[i].letter + '1'
                    TokenRecord[i].order = 4
                    i += 1
                    TokenRecord.append(Token('*',2))
                    i += 1
                    TokenRecord.append(Token(lttr,5))
                #二つ前が数値または）
                elif TokenRecord[i-1].order in (4,0):
                    if TokenRecord[i-1].letter[-1] == '.':
                        raise Exception('「 ' + TokenRecord[i].letter + ' 」の直前に小数点がきています。')
                    #トークンに分ける
                    i += 1
                    TokenRecord.append(Token(lttr,5))
                else:
                    raise Exception('+ または - が連続しています。')
            # ) 括弧
            elif TokenRecord[i].order == 0:
                #raise Exception('「 ）」と「（ 」の間に演算子を挿入してください。')
                i += 1
                TokenRecord.append(Token('*',2))
                i += 1
                TokenRecord.append(Token(lttr,5))
            #直前が+-以外の演算子・括弧（
            elif TokenRecord[i].order in (5,3,2,-1):
                #トークンに分ける
                i += 1
                TokenRecord.append(Token(lttr,5))
            #
            elif TokenRecord[i].order == 4:
                if TokenRecord[i].letter[-1] == '.':
                    raise Exception('「（ 」の直前に小数点がきています。')
                else:
                    raise Exception('「（ 」の直前に数値がきています。')
        elif lttr == '^':
            if TokenRecord[i].order == 3:
                raise Exception('べき乗が連続しています。')
            elif TokenRecord[i].order in (2,1):
                raise Exception('演算子が連続しています。')
            elif TokenRecord[i].order == 5:
                raise Exception('括弧「（ 」の直後にべき乗がきています。')
            elif TokenRecord[i].order == -1:
                raise Exception('式の先頭にべき乗がきています。')
            else:
                if TokenRecord[i].letter[-1] == '.':
                    raise Exception('「 ＾ 」の直前に小数点がきています。')
                i += 1
                TokenRecord.append(Token(lttr,3))
        elif lttr in '*/%':
            if TokenRecord[i].order in (3,2,1):
                raise Exception('演算子が連続しています。')
            if TokenRecord[i].order == 5:
                raise Exception('括弧「（ 」の直後に演算子がきています。')
            if TokenRecord[i].order == -1:
                raise Exception('式の先頭に演算子がきています')
            else:
                if TokenRecord[i].letter[-1] == '.':
                    raise Exception('「 ' + lttr + ' 」の直前に小数点がきています。')
                i += 1
                TokenRecord.append(Token(lttr,2))
        #+-はすべて
        elif lttr in '+-':
            i += 1
            TokenRecord.append(Token(lttr,1))
        elif lttr == ')':
            if TokenRecord[i].order in (3,2,1):
                raise Exception('括弧「 ）」の直前に演算子がきています。')
            if TokenRecord[i].order == 5:
                raise Exception('括弧「（ 」の直後に「 ）」がきています。')
            if TokenRecord[i].order == -1:
                raise Exception('式の先頭に閉じ括弧「 ）」がきています。')
            #数値と）は計算可
            else:
                if TokenRecord[i].letter[-1] == '.':
                    raise Exception('「 ' + lttr + ' 」の直前に小数点がきています。')
                i += 1
                TokenRecord.append(Token(lttr,0))
        else:
            raise Exception('数式の中に文字、空白、等号 = などが含まれています。',lttr)
            
    if len(TokenRecord) == 1:
        raise Exception('計算式がありません。')

    if TokenRecord[i].order not in (4,0):
        raise Exception('式の最後が演算子、または「（ 」です。')

    #与えられた式と分割したトークンをトークンをprintして確認
    print_Token(TokenRecord,expression)

    return  TokenRecord



In [39]:
#get_token_list関数のチェック
def print_Token(TokenRecord: 'list',expression: 'str'):
    """分割したTokenををprintして確認する

    :param TokenRecord: Tokenクラスのリスト
    :type TokenRecord: list
    :param exp: 式の文字列
    :type exp: str
    """

    print('式とトークン')
    print(expression)
    for token in TokenRecord:
        print(token.letter,'_',sep="", end="")
    print('')

exp = '1.09+(-29.0*(.30+4)/2*(3-1.6))-1-1'
exp = '-1-1-(-1)'
# exp = '1+2*3-10/2'
exp = '3-(-1)'
exp = '3*-(1+2)'

# print_Token(get_token_list(exp),exp)

In [ ]:
#get_token_list関数のチェック
ret = get_token_list(exp)

for le in ret:
    print(le.letter, le.order)

In [41]:

def check_parenthesis(token_record: 'list')-> 'bool':
    '''括弧のチェック
    '''
    """括弧のチェック

    :param token_record: Tokenクラスのリスト
    :type token_record: list
    :return: 括弧チェック結果の真偽値
    :rtype: bool
    """

    #番兵
    parenthesis = "("
    l = 0 #括弧の整合性カウンタ　(のときカウントアップ、)の時カウントダウン
    m = 0 #括弧の総数カウンタ

    try:

        for tkn in token_record:
            if tkn.letter == parenthesis:
                if parenthesis == "(":
                    l += 1                       #括弧整合性カウンタカウントアップ
                elif parenthesis == ")":
                    l -= 1                       #括弧整合性カウンタカウントダウン
                m += 1                           #括弧数カウンタ
            elif tkn.letter != parenthesis:
                if tkn.letter == "(":
                    parenthesis = "("
                    l += 1                       #括弧整合性カウンタカウントアップ
                    m += 1                       #括弧数カウンタ
                elif tkn.letter == ")":
                    if m == 0:
                        raise Exception('閉じカッコから始まっています。')
                    parenthesis = ")"
                    l -= 1                       #括弧整合性カウンタカウントダウン
                    m += 1                       #括弧数カウンタ
            else:
                ...

        print('括弧チェック', m, l)

        if m % 2 == 1: #括弧の数が奇数
            raise Exception('括弧の総数が奇数です。')

        if m != 0 and l != 0: #括弧整合性チェック
            raise Exception('「(」と「)」の数が合いません。')

    except Exception as exc:
        print(exc)
        return False

    else:
        return True
    

In [ ]:
#check_parenthesis関数のチェック
exp = '1.09+(-29.0*(.30+4)/2*(3-1.6))-1-1'
# exp = '-1-1-(-1)'
# exp = '1+2*3-109/2'

check_parenthesis(get_token_list(exp))

0 0


In [42]:
import collections
import traceback

def calc(expression: 'str')-> 'float':
    """四則演算、剰余、冪乗で構成される数式を計算する関数

    :param expression: 四則演算、剰余、冪乗で構成される数式の文字列
    :type expression: str
    :return: 演算結果
    :rtype: float
    """


    #演算子用スタック
    ope = collections.deque()
    #数値用スタック
    v = collections.deque()
    #番兵
    ope.append(Token('',-1)) 

    try:

        #トークンに分ける
        tokens = get_token_list(expression)
        #括弧チェック
        if not check_parenthesis(tokens):
            raise Exception('括弧チェックで不整合が検出されました」。')

        for token in tokens:
            if token.order == -1:
                #番兵の読み飛ばし
                ...
            elif token.order == 4:
                #数値の格納
                v.append(float(token.letter))
            else:
                #直前の演算子より優先順位が低い演算子の場合、演算実行。
                while token.order <= ope[-1].order and ope[-1].letter != '(':
                    right_operand = v.pop()
                    left_operand = v.pop()
                    v.append(operate(left_operand, right_operand, ope.pop().letter))

                #直前の演算子より優先順位が高い演算子の場合、演算子をスタックに積む。
                if token.letter != ')':
                    ope.append(token)

                #「（」を取り除く
                else:
                    ope.pop()

        #演算子スタックが空になるまで演算実行
        while ope[-1].order != -1:
            right_operand = v.pop()
            left_operand = v.pop()
            v.append(operate(left_operand, right_operand, ope.pop().letter)) 

    except Exception as exc:
        print(exc)
        print(traceback.format_exc())
        return None    

    else:
        return v[0]


In [44]:
#いろいろな式で計算を実行してみる
exp = '1+(2+(4+5))'
exp = '1.0+(-20.0*(.50+4)/2*(3-1.8))-1-1'
exp = '3-(-1)'
exp = '1+2*3-10/2'
exp = '3*-(1+2)'
exp = '10*2^3/20%10'
exp = '10*2^3/(22%10)'
# exp = '(1+2)(3+4)'
# exp = '1+2*3+2/0'
exp = '()'
exp = '(2*((1+3))'

calc(exp)

式とトークン
(2*((1+3))
_(_2_*_(_(_1_+_3_)_)_
括弧チェック 5 1
括弧の総数が奇数です。
括弧チェックで不整合が検出されました」。
Traceback (most recent call last):
  File "<ipython-input-42-86790ad6d5e5>", line 27, in calc
    raise Exception('括弧チェックで不整合が検出されました」。')
Exception: 括弧チェックで不整合が検出されました」。



In [43]:
#テスト
import unittest

class TestCalc(unittest.TestCase):
    
    def test(self,solution):
        self.assertEqual(solution('1+(2+(4+5))'),12)
        self.assertEqual(solution('1.0+(-20.0*(.50+4)/2*(3-1.8))-1-1'),-55)
        self.assertEqual(solution('3-(-1)'),4)
        self.assertEqual(solution('1+2*3-10/2'),2)
        self.assertEqual(solution('3*-(1+2)'),-9)
        self.assertEqual(solution('10*2^3/20%10'),4)
        self.assertEqual(solution('10*2^3/(22%10)'),40)
        self.assertEqual(solution('(1+2)(3+4)'),21)
        
        print ('Passed all tests.')

t = TestCalc()

t.test(calc)



式とトークン
1+(2+(4+5))
_1_+_(_2_+_(_4_+_5_)_)_
括弧チェック 4 0
二項演算 4.0 + 5.0
二項演算 2.0 + 9.0
二項演算 1.0 + 11.0
式とトークン
1.0+(-20.0*(.50+4)/2*(3-1.8))-1-1
_1.0_+_(_-20.0_*_(_.50_+_4_)_/_2_*_(_3_-_1.8_)_)_-_1_-_1_
括弧チェック 6 0
二項演算 0.5 + 4.0
二項演算 -20.0 * 4.5
二項演算 -90.0 / 2.0
二項演算 3.0 - 1.8
二項演算 -45.0 * 1.2
二項演算 1.0 + -54.0
二項演算 -53.0 - 1.0
二項演算 -54.0 - 1.0
式とトークン
3-(-1)
_3_-_(_-1_)_
括弧チェック 2 0
二項演算 3.0 - -1.0
式とトークン
1+2*3-10/2
_1_+_2_*_3_-_10_/_2_
括弧チェック 0 0
二項演算 2.0 * 3.0
二項演算 1.0 + 6.0
二項演算 10.0 / 2.0
二項演算 7.0 - 5.0
式とトークン
3*-(1+2)
_3_*_-1_*_(_1_+_2_)_
括弧チェック 2 0
二項演算 3.0 * -1.0
二項演算 1.0 + 2.0
二項演算 -3.0 * 3.0
式とトークン
10*2^3/20%10
_10_*_2_^_3_/_20_%_10_
括弧チェック 0 0
二項演算 2.0 ^ 3.0
二項演算 10.0 * 8.0
二項演算 80.0 / 20.0
二項演算 4.0 % 10.0
式とトークン
10*2^3/(22%10)
_10_*_2_^_3_/_(_22_%_10_)_
括弧チェック 2 0
二項演算 2.0 ^ 3.0
二項演算 10.0 * 8.0
二項演算 22.0 % 10.0
二項演算 80.0 / 2.0
式とトークン
(1+2)(3+4)
_(_1_+_2_)_*_(_3_+_4_)_
括弧チェック 4 0
二項演算 1.0 + 2.0
二項演算 3.0 + 4.0
二項演算 3.0 * 7.0
Passed all tests.
